# Auto Encoders

## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
print(torch.__version__)

2.5.1


## Importing Dataset

In [3]:
movies = pd.read_csv("ml-1m/movies.dat", sep="::", header=None, engine="python", encoding="latin-1")
users = pd.read_csv("ml-1m/users.dat", sep="::", header=None, engine="python", encoding="latin-1")
# index::userid::moviesid::rating::timestamp
ratings = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None, engine="python", encoding="latin-1")

## Preparing the training set and test set

In [4]:
training_set = pd.read_csv("ml-100k/u1.base", delimiter='\t')
training_set = np.array(training_set, dtype="int")

test_set = pd.read_csv("ml-100k/u1.test", delimiter='\t')
test_set = np.array(test_set, dtype="int")

## Getting the number of users and movies

In [5]:
nb_users = int(max(max(training_set[:, 0]), max(training_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(training_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns

In [6]:
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors

In [7]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network

In [10]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()        # full connection object
        # this second value is number of nodes in first hidden layer ( u need to change it until got a good results :P )
        self.fc1 = nn.Linear(nb_movies, 20)
        # second layer
        self.fc2 = nn.Linear(20, 10)
        # decode layer 1
        self.fc3 = nn.Linear(10, 20)
        # last layer
        self.fc4 = nn.Linear(20, nb_movies)
        
        # activation function
        self.activation = nn.Sigmoid()
    
    # x: input vector
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [12]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr= 0.01, weight_decay=0.5)

# Training the SAE

In [14]:
nb_epoch = 200

for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.0
    for user in range(nb_users):
        input = Variable(training_set[user]).unsqueeze(0)
        target = input.clone()
        # take user that rate at least one movie
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.requires_grad = False
            output[ target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 10e-10)
            # direction
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.0
            # the amount
            optimizer.step()

    print(f'epoch {epoch}, loss: {train_loss / s}')

epoch 1, loss: 1.7649396793227028
epoch 2, loss: 1.0963044769537145
epoch 3, loss: 1.0532891870019543
epoch 4, loss: 1.038386608166714
epoch 5, loss: 1.0309419714735741
epoch 6, loss: 1.0266354142213767
epoch 7, loss: 1.0239470505780046
epoch 8, loss: 1.022075806941828
epoch 9, loss: 1.020724849243325
epoch 10, loss: 1.0196138553415903
epoch 11, loss: 1.018996906047347
epoch 12, loss: 1.0183167442264742
epoch 13, loss: 1.0179417574507719
epoch 14, loss: 1.0173647264412051
epoch 15, loss: 1.0172546045104425
epoch 16, loss: 1.0169494188770156
epoch 17, loss: 1.0167892211837999
epoch 18, loss: 1.0163996807381537
epoch 19, loss: 1.0163457041278539
epoch 20, loss: 1.0161120915962398
epoch 21, loss: 1.0160333928084795
epoch 22, loss: 1.0160597672517395
epoch 23, loss: 1.0159928438744354
epoch 24, loss: 1.0159379548190388
epoch 25, loss: 1.015713117002033
epoch 26, loss: 1.015488205909989
epoch 27, loss: 1.0153268591871956
epoch 28, loss: 1.014900085776191
epoch 29, loss: 1.0136308345372662
e

# Testing the SAE

In [17]:
test_loss = 0
s = 0.0
for user in range(nb_users):
    input = Variable(training_set[user]).unsqueeze(0)
    target = Variable(test_set[user]).unsqueeze(0)
    # take user that rate at least one movie
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.requires_grad = False
        output[ target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 10e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.0

print(f'test loss: {test_loss / s}')

test loss: 0.9490311446100926
